# Regression model with Keras

### Part C. Building a baseline model with normalized data and increased number of epochs

Importing necessary libraries and loading data

In [1]:
import pandas as pd
import numpy as np

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [6]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

##### Now we normalize data

In [8]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Installing tenzor flow and Keras

In [9]:
pip install keras-tuner --upgrade

Requirement already up-to-date: keras-tuner in c:\users\sdovl\anaconda3\lib\site-packages (1.0.3)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import keras_tuner as kt
from tensorflow import keras

Creating regression model

In [11]:
from keras.models import Sequential
from keras.layers import Dense

In [12]:
n_cols = predictors_norm.shape[1] 

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [14]:
# build the model
model = regression_model()

###### 1. splitting data on train and test

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

###### 2.training model on train data

In [16]:
# fit the model
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
23/23 [==============================] - 17s 1ms/step - loss: 1484.1779
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 1526.7689
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 1470.1725
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 1429.0283
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 1430.2052
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 1388.0955
Epoch 7/100
23/23 [==============================] - 0s 1ms/step - loss: 1373.8994
Epoch 8/100
23/23 [==============================] - 0s 2ms/step - loss: 1331.4997
Epoch 9/100
23/23 [==============================] - 0s 1ms/step - loss: 1317.4044
Epoch 10/100
23/23 [==============================] - 0s 1ms/step - loss: 1232.6532
Epoch 11/100
23/23 [==============================] - 0s 1ms/step - loss: 1260.1967
Epoch 12/100
23/23 [==============================] - 0s 2ms/step - loss: 1265.5352


###### 3. Evaluating model on test data

In [17]:
loss_val = model.evaluate(X_test, y_test)
loss_val

10/10 [==============================] - 0s 887us/step - loss: 147.5383


147.5382843017578

In [18]:
y_pred = model.predict(X_test)

###### computing mse between the predicted and actual values

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mse = mean_squared_error(y_test, y_pred)
mse

147.53826928677967

###### 4. Repeating steps 1-3 50 times and create list of 50 mse

In [21]:
number_mean_squared_errors = 50
epochs = 100
mean_squared_errors = []
for i in range(0, number_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    loss_val = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mse)
    print(" mse "+str(i+1)+": "+str(mse))


 mse 1: 91.56003303612049
 mse 2: 77.58439686376583
 mse 3: 50.64197214115855
 mse 4: 48.859186163050154
 mse 5: 46.30601894781898
 mse 6: 49.05289757100195
 mse 7: 51.99783296242934
 mse 8: 42.76368212971341
 mse 9: 44.32128855729084
 mse 10: 44.45585992614939
 mse 11: 43.7803652206154
 mse 12: 41.193083954526266
 mse 13: 49.84376874634021
 mse 14: 47.98162540064651
 mse 15: 43.133859487392066
 mse 16: 40.01078983136292
 mse 17: 42.42432605005018
 mse 18: 41.6719148301735
 mse 19: 39.63834432602872
 mse 20: 42.505877772158044
 mse 21: 40.1874300978223
 mse 22: 40.779206220449694
 mse 23: 37.837201255799926
 mse 24: 41.44766866871553
 mse 25: 43.96085502122872
 mse 26: 42.77183995416366
 mse 27: 41.25104083448739
 mse 28: 40.36510540676092
 mse 29: 45.94854657558298
 mse 30: 42.50792401093534
 mse 31: 41.19212060480029
 mse 32: 37.736989950630644
 mse 33: 38.14907388126973
 mse 34: 44.09267390332356
 mse 35: 41.814620778450156
 mse 36: 46.65954115765795
 mse 37: 42.544256262220536
 mse

###### 5.Finding mean and variance of MSE

In [22]:
mse = np.array(mse)
mean = np.mean(mse)
standard_deviation = np.std(mse)

print('\n')
print("The mean and standard deviation of " +str(number_mean_squared_errors) + " mean squared errors of normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))



The mean and standard deviation of 50 mean squared errors of normalized data. Total number of epochs for each training is: 100

Mean: 48.49775711226164
Standard Deviation: 0.0


## We can notice that mean of mean squred errors is increased when we added number of epoches and difference between predicted and actual value is greater than in part B, but nevertheless better in comparing with part A with not normalized data.